In [6]:
import graphlab as gl
import pickle
import pandas as pd
import numpy as np
from collections import Counter

In [7]:
data_items = pickle.load(open("/Users/marvinbertin/Github/family_style_chat_bot/data/user_by_cuisine_by_dish_ratings.pkl", 'rb'))
data_cuisine = pickle.load(open("/Users/marvinbertin/Github/family_style_chat_bot/data/user_by_cuisine_ratings.pkl", 'rb'))

In [8]:
df_cuisine = pd.DataFrame(data_cuisine)
df_cuisine.head()

,user_id,item_id,rating
0,Brian,China,2
1,David,China,0.5
3,Jon,China,4
4,Marvin,China,0.5
6,Anne,China,2.5


In [9]:
df_cuisine.pivot(index='user_id', columns="item_id")

rating                                                            \
item_id   China France India Indonesia Italy Japan Mexico Pizza Singapore   
user_id                                                                     
Anne        2.5    1.5     4      None     4     4    1.5     5         4   
Bob        None   None  None      None   2.5     2      3   3.5       2.5   
Brian         2    3.5  None      None   4.5     5   None  None         5   
David       0.5   None  None         5   3.5   1.5   None  None       1.5   
Eugene     None   None  None       0.5     2  None   None     1         2   
Jon           4      1     4       2.5   0.5     3   None  None         5   
Leah          4   None     1      None  None  None    2.5     5       1.5   
Marvin      0.5   None   4.5       4.5  None  None   None     2      None   
Michelle    4.5      1  None         5   4.5  None   None   1.5      None   
Samatha       2      3     5      None  None   2.5   None  None       3.5   
Sarah       0.5   None   1.5      None  None  None   None     1         3   

                           
item_id  Thailand Vietnam  
user_id                    
Anne         None       5  
Bob          None     1.5  
Brian         3.5    None  
David         3.5       2  
Eugene          1    None  
Jon          None     2.5  
Leah         None    None  
Marvin        1.5    None  
Michelle     None     1.5  
Samatha       4.5     2.5  
Sarah           4     2.5

In [10]:
data_items.keys()

['Italy',
 'Mexico',
 'Indonesia',
 'India',
 'Singapore',
 'France',
 'Vietnam',
 'China',
 'Thailand',
 'Japan',
 'Pizza']

In [26]:
class group_recommender(object):
    def __init__(self, cuisine_sf, dict_cuisine_items):
        self.cuisine = gl.SFrame(cuisine_sf)
        self.cuisine_items = dict_cuisine_items
    
    def recommend(self, group_list):
        group_name = "_".join(group_list)
        
        sf_avg_user = self.cuisine.filter_by(group_list, "user_id") \
                                  .groupby(key_columns='item_id',
                                           operations={'rating': gl.aggregate.MEAN('rating')})  
        
        sf_avg_user.add_column(gl.SArray([group_name] * len(sf_avg_user)), "user_id")
#         print sf_avg_user
        
        sf_new = self.cuisine.append(sf_avg_user)
        
        model = gl.recommender.create(sf_new, target='rating')
    
        results = model.recommend([group_name], exclude_known=False)
#         print results
        result_cuisine = results["item_id"][:3]
    
        option_list = []
        for cuisine in result_cuisine:
            
            sf_items = gl.SFrame(self.cuisine_items[cuisine])
            model_items = gl.recommender.create(sf_items, target='rating')

            results_items = model_items.recommend(group_list, exclude_known=False, k = 2)
    #         print results_items
                
            if cuisine == "Pizza":
                group_size = len(group_list)
                num_pizza = int(group_size / 1.5)
                item_results = [item for item, count in Counter(results_items["item_id"]).most_common()][:num_pizza]
                option_list.append(("Pizza Party!", item_results))
        
            else:
                group_size = len(group_list)
                item_results = [item for item, count in Counter(results_items["item_id"]).most_common()][:group_size]
                option_list.append((cuisine, item_results))

        return option_list

In [44]:
group_list = np.random.choice(df_cuisine["user_id"].unique(), size = 2, replace=False)

print group_list
model = group_recommender(df_cuisine, data_items)
result = model.recommend(group_list)

['Eugene' 'Sarah']


Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 78 observations with 12 users and 11 items.

Data prepared in: 0.005478s

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 1.98392

Final training RMSE: 0.712421

In [45]:
result

[('Singapore', ['Singapore Chili Crabs', 'Sesame Oil Chicken Wings']),
 ('Thailand', ['Thai Ginger Soup', 'Thai Hout and Sour Soup']),
 ('Italy', ['Creamy Italian White Bean Soup', 'Pork Chops Italiano'])]